In [1]:
from classes.utils import *
import requests # Import the requests module
import pandas as pd
import sys
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
import json
from datetime import datetime

from classes.tournaments import Tournament
from classes.teams import Team
from classes.players import Player

ImportError: cannot import name 'Tournament' from partially initialized module 'classes.tournaments' (most likely due to a circular import) (e:\Programações\sofascoreBR\classes\tournaments.py)

In [2]:
# PEGAR JOGADORES DO TIME ATUALIZADO POR ANO
# PEGAR JOGADORES VIA LINEUPS?
# METRICAS JOGADORES POR JOGO E POR CAMPEONATO

Made changes.

In [3]:
#brasileirao = Tournament(id = 325, year = 2024)
#cruzeiro = Team(id = 1954, tournament_id = None, season_id = None, name=None)
#MP = Player(id = 377238, name=None, shortName=None, teamId=None, tournamentId=None, seasonId=None, position=None)

'M. Pereira'